In [ ]:
from __future__ import print_function, division
from tensorflow import keras
import tensorflow as tf

from keras.layers import *
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
import keras.backend as K

from math import log2
import numpy as np
from random import randint

import matplotlib.pyplot as plt

In [ ]:
# Константы
img_side = 204

# Разбиваем датасет на тренировочную группу и группу валидации
def init_data_with_batch_size(batch_size, dataset="flowers"):
    """Чтобы использовать "new_flowers" (расширенный датасет) надо запустить increasing_data.py"""
    train_data = keras.preprocessing.image_dataset_from_directory(
        dataset,
        image_size=(img_side, img_side),
        label_mode="int",  # Тут не "categorical" т.к. мы испольлзауем Embedding
        shuffle=True,
        batch_size=batch_size,
    )

    # Добавляем лейблы (т.к. у нас Cgan) и нормализуем в [-1; 1], т.к. юзаем tanh
    # (не sigmoid, а tanh, т.к. с ним градиент не затухает)
    x = np.array([i[0]/127.5 -1 for i in train_data][:-1])
    y = np.array([i[1] for i in train_data][:-1])

    return x, y

In [ ]:
data = init_data_with_batch_size(1, "flowers")

cgan.train(data, 16, 20000, sample_interval=500)

In [ ]:
def show_row_images(raw_data):
    # Ограничиваемся только 32 восстановленными изображениями (чтобы считать меньше)
    data = np.array([i[0][0] for count, i in enumerate(raw_data) if count < 32])
    labels = np.array([i[1][0] for count, i in enumerate(raw_data) if count < 32])
    _, _, encoded = encoder.predict([data, labels], verbose=False)
    generated_images = decoder.predict([encoded, labels], verbose=False)

    num_images = 4

    plt.figure(figsize=(20, 11))

    for _ in range(num_images):
        random_num = randint(0, 32-1)

        # Оригинальное изображение
        plt.subplot(2, num_images, _ + 1)
        plt.imshow(data[random_num])
        plt.gray()
        plt.title("Train")
        plt.axis("off")

        # Сгенерированное изображение
        plt.subplot(2, num_images, _ + num_images + 1)
        plt.imshow(generated_images[random_num])
        plt.gray()
        plt.title("Generated")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

raw_data = init_data_with_batch_size(1, "flowers")
for _ in range(3):
    show_row_images(raw_data)

In [ ]:
d = init_data_with_batch_size(1, "flowers")
data = np.array([next(iter(d))[0][0] for _ in range(1000)])
labels = np.array([next(iter(d))[1][0] for _ in range(1000)])

_, _, all_hidden_data = encoder.predict([data, labels])
mean, std = np.mean(all_hidden_data), np.std(all_hidden_data)
print("mean (ideal: 0):", mean)
print("std  (ideal: 1):", std)

for _ in range(4):
    num_images = 4

    noise = np.random.normal(mean, std, [num_images*2, hidden_units])
    label = np.array([ keras.utils.to_categorical(np.random.randint(0, 5), 5)
                       for _ in range(num_images*2)])
    generated_images = np.array(decoder.predict([noise, label], verbose=False))

    plt.figure(figsize=(20, 11))

    for i in range(num_images):
        # Оригинальное изображение
        plt.subplot(2, num_images, i + 1)

        # Переводим в промежуток [0; 1]
        plt.imshow(generated_images[i + num_images])
        plt.axis("off")

        # Сгенерированное изображение
        plt.subplot(2, num_images, i + num_images + 1)
        plt.imshow(generated_images[i])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
"""Выводим Архитектуру"""
encoder_img = tf.keras.utils.plot_model(encoder, to_file="encoder.png", show_shapes=False, show_layer_names=False,
                                        dpi=128, show_layer_activations=False)

decoder_img = tf.keras.utils.plot_model(decoder, to_file="decoder.png", show_shapes=False, show_layer_names=False,
                                        dpi=128, show_layer_activations=False)

In [ ]:
cgan.save_weights("cgan")
# cgan.load_weights("cgan")